Decision tree classifier using the dataset

Import statements

In [89]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score

Importing the dataset

In [90]:
df = pd.read_csv("./CleanedData.csv", index_col=0)
columns = df.columns
print(df.head())

   GRE Score  TOEFL Score  University Rating  SOP  LOR   CGPA  Research  \
0      337.0        118.0                  4  4.5   4.5  9.65         1   
1      324.0        107.0                  4  4.0   4.5  8.87         1   
2      316.0        104.0                  3  3.0   3.5  8.00         1   
3      322.0        110.0                  3  3.5   2.5  8.67         1   
4      314.0        103.0                  2  2.0   3.0  8.21         0   

   SES Percentage  Asian  african american  latinx  white  Chance of Admit  
0              12      1                 0       0      0             0.92  
1              11      0                 0       1      0             0.76  
2              78      0                 0       1      0             0.72  
3              77      0                 0       0      1             0.80  
4               1      0                 1       0      0             0.65  


defining x

In [91]:
x = df[columns[0:-1]]


Since decision trees are reliant on the fact that the classification is binary, I need to set a threshold for what
chance of admit means the student will be admitted.

In [92]:
median_value = df["Chance of Admit"].median()
print(median_value)

0.73


Since the median chance of admission is 73%, I will use that as the threshold. Any chance of admission >= 0.73 will be
set to one, and any student < 0.73 will be set to 0

In [93]:
for i in range(len(df["Chance of Admit"])):
    if df.loc[i, "Chance of Admit"] >= 0.73:
        df.loc[i, "Chance of Admit"] = 1
    else:
        df.loc[i, "Chance of Admit"] = 0
print(df["Chance of Admit"])

0      1.0
1      1.0
2      0.0
3      1.0
4      0.0
      ... 
351    1.0
352    1.0
353    1.0
354    0.0
355    1.0
Name: Chance of Admit, Length: 356, dtype: float64


Defining y as the chance of admit column

In [94]:
y = df["Chance of Admit"]

split the data into training and testing

In [95]:
x_train, x_test, y_train, y_test = train_test_split(x, y, shuffle=True, random_state=1)
print(x_train.head())
print(y_train.head())

     GRE Score  TOEFL Score  University Rating  SOP  LOR   CGPA  Research  \
23       300.0         97.0                  2  3.0   3.0  8.10         1   
320      338.0        115.0                  5  4.5   5.0  9.23         1   
213      328.0        110.0                  4  4.0   2.5  9.02         1   
51       307.0        101.0                  3  4.0   3.0  8.20         0   
92       301.0        107.0                  3  3.5   3.5  8.34         1   

     SES Percentage  Asian  african american  latinx  white  
23               93      0                 0       1      0  
320              19      0                 0       0      1  
213              15      0                 0       0      1  
51               94      0                 0       0      1  
92               52      0                 0       1      0  
23     0.0
320    1.0
213    1.0
51     0.0
92     0.0
Name: Chance of Admit, dtype: float64


Using grid search to find the correct hyperparameters

In [96]:
criteria = ["gini", "entropy"]
max_depths = [i for i in range(3, 11)]
min_samples_leaves = [i for i in range(1, 15)]

model_scores = []

for c in criteria:
    for depth in max_depths:
        for leaf_size in min_samples_leaves:
            model = DecisionTreeClassifier(criterion=c, max_depth=depth, min_samples_leaf=leaf_size)
            model.fit(x_train, y_train)
            score = model.score(x_test, y_test)
            model_scores.append(score)

            print("Criteria: " + c, "\nMax Depth: " + str(depth) + "\nMin Leaf Size: " + str(leaf_size) + "\nScore: " + str(score))
            print()

Criteria: gini 
Max Depth: 3
Min Leaf Size: 1
Score: 0.8314606741573034

Criteria: gini 
Max Depth: 3
Min Leaf Size: 2
Score: 0.8314606741573034

Criteria: gini 
Max Depth: 3
Min Leaf Size: 3
Score: 0.8314606741573034

Criteria: gini 
Max Depth: 3
Min Leaf Size: 4
Score: 0.8314606741573034

Criteria: gini 
Max Depth: 3
Min Leaf Size: 5
Score: 0.8314606741573034

Criteria: gini 
Max Depth: 3
Min Leaf Size: 6
Score: 0.8314606741573034

Criteria: gini 
Max Depth: 3
Min Leaf Size: 7
Score: 0.8202247191011236

Criteria: gini 
Max Depth: 3
Min Leaf Size: 8
Score: 0.8202247191011236

Criteria: gini 
Max Depth: 3
Min Leaf Size: 9
Score: 0.8202247191011236

Criteria: gini 
Max Depth: 3
Min Leaf Size: 10
Score: 0.8202247191011236

Criteria: gini 
Max Depth: 3
Min Leaf Size: 11
Score: 0.8089887640449438

Criteria: gini 
Max Depth: 3
Min Leaf Size: 12
Score: 0.8089887640449438

Criteria: gini 
Max Depth: 3
Min Leaf Size: 13
Score: 0.8089887640449438

Criteria: gini 
Max Depth: 3
Min Leaf Size: 14


We found that a tree with the Gini criteria and a max depth of 4 and leaf size of 4 gave us an score of 84%. In general,
however, the range of accuracy scores for almost every model in the grid search has been between 79% and 83% inclusive.
This means the decision tree is fairly robust against hyperparameter changes.


We think we can do better. Now we will use the random forest to determine if that can provide better accuracy scores
than a standalone tree.

confirming the data is still in tact

In [98]:
print(x_train)
print(y_train)
print(x_test)
print(y_test)

     GRE Score  TOEFL Score  University Rating  SOP  LOR   CGPA  Research  \
23       300.0         97.0                  2  3.0   3.0  8.10         1   
320      338.0        115.0                  5  4.5   5.0  9.23         1   
213      328.0        110.0                  4  4.0   2.5  9.02         1   
51       307.0        101.0                  3  4.0   3.0  8.20         0   
92       301.0        107.0                  3  3.5   3.5  8.34         1   
..         ...          ...                ...  ...   ...   ...       ...   
203      326.0        111.0                  5  4.5   4.0  9.23         1   
255      300.0        102.0                  2  1.5   2.0  7.87         0   
72       299.0         97.0                  3  5.0   3.5  7.66         0   
235      308.0        108.0                  4  4.5   5.0  8.34         0   
37       329.0        114.0                  5  4.0   5.0  9.30         1   

     SES Percentage  Asian  african american  latinx  white  
23           

We will now run grid search to determine the best hyperparameters for a random forest are.

In [103]:
n_ests = [i for i in range(50, 500, 50)]
criteria = ["gini", "entropy"]
accuracies = []

for n in n_ests:
    for c in criteria:
        model = RandomForestClassifier(criterion=c, n_estimators=n, bootstrap=True)
        model.fit(x_train, y_train)
        score = model.score(x_test, y_test)
        features = model.feature_importances_
        print("Criteria: " + c +"\nNumber of trees: " + str(n) + "\nScore: " + str(score) + "\nFeatures: " + str(features))
        print()


Criteria: gini
Number of trees: 50
Score: 0.8202247191011236
Features: [0.19699793 0.11429686 0.0845743  0.0986928  0.06896064 0.27497449
 0.08070116 0.05610201 0.00201313 0.00916008 0.00372534 0.00980127]

Criteria: entropy
Number of trees: 50
Score: 0.8314606741573034
Features: [0.15759239 0.12330531 0.08912162 0.09377104 0.08382883 0.29842692
 0.06170553 0.06557811 0.0019483  0.01031946 0.00470282 0.00969968]

Criteria: gini
Number of trees: 100
Score: 0.8202247191011236
Features: [0.20143844 0.13741945 0.08173773 0.10336294 0.08752444 0.2366771
 0.05991958 0.06423719 0.00222499 0.00981049 0.00438476 0.0112629 ]

Criteria: entropy
Number of trees: 100
Score: 0.8089887640449438
Features: [0.21170072 0.10523958 0.08906135 0.10944229 0.06859987 0.25165514
 0.06125271 0.06907851 0.0027658  0.01289939 0.00483612 0.01346851]

Criteria: gini
Number of trees: 150
Score: 0.8202247191011236
Features: [0.23681653 0.1072696  0.0879072  0.10923649 0.07178814 0.21754155
 0.07664059 0.0670587  0.0

We found that the robustness of the model increased with random forest, meaning that the accuracy score was held between
80% and 83%. However, with the hyperparameters correct with the decision tree, we can achieve an accuracy higher than
the random forest. The robustness might be very conducive to using data such as this, though.